## With input file

In [51]:
path = '/Users/huxinyu/Desktop/UROP2023/IMG_9976.wav'

In [52]:
num_speakers = 2 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'tiny' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [53]:
# !pip install -q git+https://github.com/openai/whisper.git > /dev/null
# !pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding( 
    "speechbrain/spkrec-ecapa-voxceleb")

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [54]:
model = whisper.load_model(model_size)

In [55]:
result = model.transcribe(path)
segments = result["segments"]

In [56]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [57]:
audio = Audio()

# when dealing with audios with 2 channels, one can uncomment the following code.
# def segment_embedding_(segment):
#   start = segment["start"]
#   # Whisper overshoots the end timestamp in the last segment
#   end = min(duration, segment["end"])
#   clip = Segment(start, end)
#   waveforms, sample_rate = audio.crop(path, clip)
#   waveforms = waveforms[1]
#   waveforms = waveforms[None, None, :]
#   return embedding_model(waveforms)

# one-channel audio file
def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [58]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
# the detected transcript will be saved in the file transcript.txt.

def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n\n" + segment["speaker"] + ':')
  f.write(segment["text"][1:] + ' ')
f.close()